In [2]:
#install package
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import numpy as np
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [4]:
#import file for training
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [7]:
df = pd.read_csv('train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [8]:
#preprocessing
df.drop(['keyword', 'location', 'id'], axis=1, inplace=True)
stop = stopwords.words('english')
np.random.seed(19970901)

def url(text):
    stuff = re.compile(r'https?://\S+|www\.\S+')
    return stuff.sub(r'', text)

def emoji(text):
    stuff = re.compile(
        '['
        u'\U0001F600-\U0001F64F'
        u'\U0001F300-\U0001F5FF'
        u'\U0001F680-\U0001F6FF'
        u'\U0001F1E0-\U0001F1FF'
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return stuff.sub(r'', text)

def html(text):
    stuff = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(stuff, '', text)

def punctuation(text):
    stuff = str.maketrans('', '', string.punctuation)
    return text.translate(stuff)

df['text'] = df['text'].apply(lambda x: url(x))
df['text'] = df['text'].apply(lambda x: emoji(x))
df['text'] = df['text'].apply(lambda x: html(x))
df['text'] = df['text'].apply(lambda x: punctuation(x))
df['text'] = df['text'].apply(lambda x: str.lower(x))
#df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#split into train and test
#train = df.iloc[:6090, :]
#test = df.iloc[6091:, :]

In [10]:
from transformers import MarianMTModel, MarianTokenizer
target_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
target_model = MarianMTModel.from_pretrained(target_model_name)

INFO:filelock:Lock 139959032129552 acquired on /root/.cache/huggingface/transformers/fb8aafd158087ac5c00353d405eee8e51221637ecf90c3e333a72cd9d9c5c5ca.ca44de66acb7ecec9ad9bc44657f6c66b140954114f2698ae3bf4238180d6c25.lock


Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

INFO:filelock:Lock 139959032129552 released on /root/.cache/huggingface/transformers/fb8aafd158087ac5c00353d405eee8e51221637ecf90c3e333a72cd9d9c5c5ca.ca44de66acb7ecec9ad9bc44657f6c66b140954114f2698ae3bf4238180d6c25.lock
INFO:filelock:Lock 139959259625424 acquired on /root/.cache/huggingface/transformers/69a6c861e644687303c737f24264fcfb7aa8d15241df48fc2f8ff01f959ebe81.3de0604e0b9235d2638364f72d5a29445f57716a48b270e405b68b8845a5dee4.lock


Downloading:   0%|          | 0.00/799k [00:00<?, ?B/s]

INFO:filelock:Lock 139959259625424 released on /root/.cache/huggingface/transformers/69a6c861e644687303c737f24264fcfb7aa8d15241df48fc2f8ff01f959ebe81.3de0604e0b9235d2638364f72d5a29445f57716a48b270e405b68b8845a5dee4.lock
INFO:filelock:Lock 139959029383888 acquired on /root/.cache/huggingface/transformers/f6c3d31015446eb353b81c04428534579654df1d9cf0c557b3b53b6e4d996233.189d6e3d3d2352d6b755be23081b245cffad5bfbac6e411b9817d232d33006e4.lock


...

INFO:filelock:Lock 139959032128016 released on /root/.cache/huggingface/transformers/31cbcbb209cda19eeb0e51232ac0fd496248eb8fce6a35775bf8085237f5b82c.1a55e122dd67cdf8bb745e78cd5a5f7aeec191b6b74427efb298653782072f96.lock
INFO:filelock:Lock 139959022545232 acquired on /root/.cache/huggingface/transformers/487622d499e0c34331c81104a7228845cfa94f52aa29f7edbdceb62121bf4810.7f0f6fa8031a9d862d43b3d0f28c33c534bfbb3d867c815d5f2db2a99f6ee87b.lock


Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

INFO:filelock:Lock 139959022545232 released on /root/.cache/huggingface/transformers/487622d499e0c34331c81104a7228845cfa94f52aa29f7edbdceb62121bf4810.7f0f6fa8031a9d862d43b3d0f28c33c534bfbb3d867c815d5f2db2a99f6ee87b.lock
INFO:filelock:Lock 139959021274896 acquired on /root/.cache/huggingface/transformers/1c842cfcab680969366a746a8d6c3ac5dfbae637bdcf87cd7d50664443dcb7b6.f243601ee976d095d5a0a71dbbb207f01edc53e1b32f8c39456ffa572deb3f99.lock


Downloading:   0%|          | 0.00/312M [00:00<?, ?B/s]

INFO:filelock:Lock 139959021274896 released on /root/.cache/huggingface/transformers/1c842cfcab680969366a746a8d6c3ac5dfbae637bdcf87cd7d50664443dcb7b6.f243601ee976d095d5a0a71dbbb207f01edc53e1b32f8c39456ffa572deb3f99.lock
Some weights of MarianMTModel were not initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-ROMANCE and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
en_tokenizer = MarianTokenizer.from_pretrained(en_model_name)
en_model = MarianMTModel.from_pretrained(en_model_name)

INFO:filelock:Lock 139959013162896 acquired on /root/.cache/huggingface/transformers/fb5a688525d6c40a1f822784bba5c568a6209a7f68bbfd81ad30aee8af9fe757.2bd248ef954b136f33ba3b5f29736fc22fe21299b45a94ae1edd3136a6f1ddb2.lock


Downloading:   0%|          | 0.00/800k [00:00<?, ?B/s]

INFO:filelock:Lock 139959013162896 released on /root/.cache/huggingface/transformers/fb5a688525d6c40a1f822784bba5c568a6209a7f68bbfd81ad30aee8af9fe757.2bd248ef954b136f33ba3b5f29736fc22fe21299b45a94ae1edd3136a6f1ddb2.lock
INFO:filelock:Lock 139959032128144 acquired on /root/.cache/huggingface/transformers/c82b120b36d8b28738f1a1c0757b7f4872c909ec18453800e48e76412b2a224f.d6d474c4ca928339db0d7a9e8e4663b795225c29b46463593ab08c445ad08fbf.lock


Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

INFO:filelock:Lock 139959032128144 released on /root/.cache/huggingface/transformers/c82b120b36d8b28738f1a1c0757b7f4872c909ec18453800e48e76412b2a224f.d6d474c4ca928339db0d7a9e8e4663b795225c29b46463593ab08c445ad08fbf.lock
INFO:filelock:Lock 139959052379152 acquired on /root/.cache/huggingface/transformers/dbe77a7e08998d5dd16d296d52ac847d948ce8623f9d03521523228adef7738a.f5d5ee4d8c7ca816f3fabd88ce4c54472271b2910a3f09caf4b889aca978f38c.lock


...

INFO:filelock:Lock 139959021130448 released on /root/.cache/huggingface/transformers/eeea462f521dc2d50bc28be24a8993a6f84552c3b0cc16036733b25b00b76eff.b9af6e61147590608619b5508e4a608e12d5907e641fa73ebb9da3ea5f8c1d94.lock
INFO:filelock:Lock 139959014117200 acquired on /root/.cache/huggingface/transformers/231dc7d689812283cb707214f58bffc06d1c042753c454b4ece39778798e5c2a.768ae45466792a0f33057b9bb31f8e8c6cbdbac9d3b2fb504e57ba2ab840a489.lock


Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

INFO:filelock:Lock 139959014117200 released on /root/.cache/huggingface/transformers/231dc7d689812283cb707214f58bffc06d1c042753c454b4ece39778798e5c2a.768ae45466792a0f33057b9bb31f8e8c6cbdbac9d3b2fb504e57ba2ab840a489.lock
INFO:filelock:Lock 139959014117136 acquired on /root/.cache/huggingface/transformers/280541649472174b9a8bfa31ba0c861e806be14d2f60113b6da0d1fb5ec395bb.c7a1c260ba157336a16a3228f89a319e62669fcbb609e915f81ad0a41cde9a36.lock


Downloading:   0%|          | 0.00/312M [00:00<?, ?B/s]

INFO:filelock:Lock 139959014117136 released on /root/.cache/huggingface/transformers/280541649472174b9a8bfa31ba0c861e806be14d2f60113b6da0d1fb5ec395bb.c7a1c260ba157336a16a3228f89a319e62669fcbb609e915f81ad0a41cde9a36.lock
Some weights of MarianMTModel were not initialized from the model checkpoint at Helsinki-NLP/opus-mt-ROMANCE-en and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def translate(texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    template = lambda text: f"{text}" if language == "en" else f">>{language}<< {text}"
    src_texts = [template(text) for text in texts]

    # Tokenize the texts
    encoded = tokenizer.prepare_seq2seq_batch(src_texts)
    
    # Generate translation using model
    translated = model.generate(**encoded)

    # Convert the generated tokens indices back into text
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return translated_texts

In [13]:
def back_translate(texts, source_lang="en", target_lang="fr"):
    # Translate from source to target language
    fr_texts = translate(texts, target_model, target_tokenizer, 
                         language=target_lang)

    # Translate from target language back to source language
    back_translated_texts = translate(fr_texts, en_model, en_tokenizer, 
                                      language=source_lang)
    
    return back_translated_texts

In [14]:
df.head()
df['text'] = back_translate(list(df['text']), source_lang="en", target_lang="es")
df.head()

AttributeError: ignored

In [ ]:
#specify simple transformer model with number of epochs
model_args = ClassificationArgs(num_train_epochs = 5,learning_rate = 3e-5)
model = ClassificationModel(
    "roberta", "roberta-base", args=model_args, use_cuda=True)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
#Train model and evaluate test set
model.train_model(df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:449: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/7613 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/952 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/952 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/952 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/952 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/952 [00:00<?, ?it/s]

(4760, 0.3684720074975112)

In [ ]:
test = pd.read_csv('test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [ ]:
test.drop(['keyword', 'location'], axis=1, inplace=True)
test['text'] = test['text'].apply(lambda x: url(x))
df['text'] = df['text'].apply(lambda x: emoji(x))
test['text'] = test['text'].apply(lambda x: html(x))
test['text'] = test['text'].apply(lambda x: punctuation(x))
test['text'] = test['text'].apply(lambda x: str.lower(x))
#test['text'] = test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
predictions, raw_outputs = model.predict(list(test['text']))
print(predictions)

  0%|          | 0/3263 [00:00<?, ?it/s]

  0%|          | 0/408 [00:00<?, ?it/s]

[1 1 1 ... 1 1 1]


In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      3263 non-null   int64
 1   target  3263 non-null   int64
dtypes: int64(2)
memory usage: 51.1 KB


In [ ]:
submission['target'] = predictions
submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [ ]:
#submission.to_csv('submission.csv')
from google.colab import drive
drive.mount('/content/gdrive')
#!cp submission.csv "drive/My Drive/"

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/
! ls
#! git clone https://github.com/alainrouillon/py-googletrans.git
%cd ./py-googletrans
! git checkout origin/feature/enhance-use-of-direct-api
! python setup.py install


[Errno 2] No such file or directory: 'gdrive/MyDrive/'
/content/gdrive/MyDrive
 ae_weights.h5
 Calculus-Early-Transcendentals.pdf
'comments_magdamobrendan_68969_1264856_Essay #1 (1).docx.gdoc'
'comments_magdamobrendan_68969_1264856_Essay #1.docx'
'comments_magdamobrendan_68969_1264856_Essay #1.docx.gdoc'
'comments_magdamobrendan_68969_1407934_Information Literacy Assignment (1).docx.gdoc'
'comments_magdamobrendan_68969_1407934_Information Literacy Assignment.docx'
'comments_magdamobrendan_68969_1407934_Information Literacy Assignment.docx.gdoc'
'comments_magdamobrendan_68969_1519454_Essay #2 (1).docx.gdoc'
'comments_magdamobrendan_68969_1519454_Essay #2.docx'
'comments_magdamobrendan_68969_1519454_Essay #2.docx.gdoc'
'CP 1.7- Ethics in Data Science.gdoc'
 DEC_model_final.h5
'Getting started.pdf'
 Lab1DataSlope.gsheet
'Magdamo HW2.pdf'
'Major Paper #1.docx'
'Major Paper #1.docx.gdoc'
'PD 2.1A - Bimonthly Project Report I.gdoc'
 py-googletrans
 Resume_updated.pdf
'Science project.gslides

In [ ]:
from google_trans_new import google_translator 

In [ ]:
from google_trans_new import google_translator  
translator = google_translator()  
translate_text = translator.translate('สวัสดีจีน',lang_tgt='en')  
print(translate_text)
#df['text'] = df['text'].apply(translator.translate, src='en', dest='sw').apply(getattr, args=('text',))
#df.head()

Hello Chinese 
